# A. Librerias y rutas

In [35]:
import datetime
import pandas as pd
import os
import time
import json
import wget
import zipfile
import requests
import platform

import datetime
import pandas as pd
from browsermobproxy import Server

from seleniumwire import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC

In [36]:
# 1. Definir rutas y sistema operativo
os_platform = platform.system().lower()

# Definir la carpeta de descargas para los archivos JSON
LOG_FILE_PATH = os.path.join("./scraping_log.txt")

# 2. Configurar rutas del proyecto según el sistema operativo
if os_platform == "darwin":
    BASE_PATH = "/Volumes/Projects/GitHubProjects/UNI-SCRAPING"
    CHROMEDRIVER_BASE_PATH = os.path.join(BASE_PATH, "chromedriver")
    BROWSERMOB_BASE_PATH = os.path.join(BASE_PATH, "browsermob-proxy-2.1.4")
elif os_platform == "linux":
    BASE_PATH = "/home/brew_test_gcp_01/Desktop/WallyScraper"
    CHROMEDRIVER_BASE_PATH = os.path.join(BASE_PATH, "chromedriver")
    BROWSERMOB_BASE_PATH = os.path.join(BASE_PATH, "browsermob-proxy-2.1.4")
elif os_platform == "windows":
    BASE_PATH = "D:\\GithubProjects\\Tesis\\Transparencia"
    CHROMEDRIVER_BASE_PATH = os.path.join(BASE_PATH, "chromedriver")
    BROWSERMOB_BASE_PATH = os.path.join(BASE_PATH, "browsermob-proxy-2.1.4")

if os_platform == "darwin":
    CHROMEDRIVER_PATH = os.path.join(CHROMEDRIVER_BASE_PATH, "darwin", "chromedriver")
    BROWSERMOB_PATH = os.path.join(BROWSERMOB_BASE_PATH, "bin", "browsermob-proxy")
elif os_platform == "linux":
    CHROMEDRIVER_PATH = os.path.join(CHROMEDRIVER_BASE_PATH, "linux", "chromedriver")
    BROWSERMOB_PATH = os.path.join(BROWSERMOB_BASE_PATH, "bin", "browsermob-proxy")
elif os_platform == "windows":
    CHROMEDRIVER_PATH = os.path.join(CHROMEDRIVER_BASE_PATH, "windows", "chromedriver.exe")
    BROWSERMOB_PATH = os.path.join(BROWSERMOB_BASE_PATH, "bin", "browsermob-proxy.bat")


# 3. Imprimir variables
print("ESTAMOS EN ---> ", os_platform)
print("CHROME_DRIVER_PATH: ", CHROMEDRIVER_PATH)
print("BROWSERMOB_PATH: ", BROWSERMOB_PATH)
print("")

ESTAMOS EN --->  darwin
CHROME_DRIVER_PATH:  /Volumes/Projects/GitHubProjects/UNI-SCRAPING/chromedriver/darwin/chromedriver
BROWSERMOB_PATH:  /Volumes/Projects/GitHubProjects/UNI-SCRAPING/browsermob-proxy-2.1.4/bin/browsermob-proxy



# B. Funciones

In [37]:
def get_chrome_driver(chromedriver_path=None, print_view=False, headless=False):
    # Iniciar el servidor proxy
    print(f"Iniciando servidor proxy en {os_platform}")
    server = Server(BROWSERMOB_PATH)
    server.start()
    proxy = server.create_proxy(params={'trustAllServers': 'true'})
    
    # Configurar las opciones de Selenium
    options = webdriver.ChromeOptions()
    options.add_argument("--no-sandbox")
    options.add_argument("--start-maximized")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--ignore-certificate-errors")
    options.add_argument("--proxy-server={0}".format(proxy.proxy))

    if print_view:
        options.add_argument('--disable-print-preview')
    
    if headless:
        options.add_argument("--headless=new")

    service = Service(executable_path=chromedriver_path)
    driver = webdriver.Chrome(service=service, options=options)

    return driver, server, proxy

In [38]:
def stop_dependencies(driver, server, proxy):
    driver.quit()
    server.stop()
    proxy.close()

In [39]:
def start_har(proxy):
    options = {
        'captureHeaders': True, 
        'captureContent': True, 
        'captureBinaryContent': True, 
        'captureCookies': True, 
        'captureHeadersSize': -1, 
        'captureMaxSize': -1, 
        'captureBinaryContentMaxLength': -1 
    }
    proxy.new_har("wally", options=options)

# C. Probar

In [40]:
driver, server, proxy = get_chrome_driver(chromedriver_path=CHROMEDRIVER_PATH, print_view=False, headless=False)

Iniciando servidor proxy en darwin


In [41]:
start_har(proxy)

In [42]:
url = 'https://www.senamhi.gob.pe/site/descarga-datos/'
driver.get(url)

time.sleep(5)
# Analizar requests capturadas
for request in driver.requests:
    if request.response:
        print(f"URL: {request.url}, Status: {request.response.status_code}")
        print(f"Response Headers: {request.response.headers}")
        print("="*50)

URL: http://clients2.google.com/time/1/current?cup2key=8:PPKW9yET3FCu99qyYcKEyK_p1-c5rtggttJmjwl3v5w&cup2hreq=e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855, Status: 200
Response Headers: Content-Type: application/json; charset=utf-8
Vary: Sec-Fetch-Dest, Sec-Fetch-Mode, Sec-Fetch-Site
X-Content-Type-Options: nosniff
x-cup-server-proof: 3046022100b43958637f08ff53e9c4a17db8ce94913bc9f23e23d1675b9b5633d34a690063022100c9634d0888c8a2e865aa7be6baa598078b6a6d4247d982d5e5c92dd991ab5192:e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855
ETag: W/"3046022100b43958637f08ff53e9c4a17db8ce94913bc9f23e23d1675b9b5633d34a690063022100c9634d0888c8a2e865aa7be6baa598078b6a6d4247d982d5e5c92dd991ab5192:e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855"
Cache-Control: no-cache, no-store, max-age=0, must-revalidate
Pragma: no-cache
Expires: Mon, 01 Jan 1990 00:00:00 GMT
Date: Wed, 12 Feb 2025 20:03:42 GMT
Content-Disposition: attachment; filename="json.txt"; filen

In [29]:
time.sleep(5)
with open('./results/html.html', 'w') as f:
    f.write(driver.page_source)

In [30]:
time.sleep(2)
har = proxy.har

with open('./results/har.json', 'w') as f:
    json.dump(har, f, indent=4)

AttributeError: 'WebDriver' object has no attribute 'requests'